# Working with Cloud (Remote) Back-ends
## So far
- We've learnt to create a synthetic dataset
- Train a logical function to make predictions from the synthetic dataset
- Test and develop an API server with Flask
- Deploy our prediction function to Heroku

## In this section
We will learn to interact with Google's Firebse and create a document collection **to demonstrate the usecase when our application needs to collect and store user data**.

## Setup and Authentication

In [1]:
import pandas as pd
import firebase_admin
from firebase_admin import credentials, firestore

#### Point to Firebase credentials

In [2]:
cred = credentials.Certificate("../../configs/ww-save-nyc-firebase-adminsdk-eh9d8-5e01de8f8c.json")

#### Initialize firebase client as admin

In [3]:
firebase_admin.initialize_app(cred,
    {
        "databaseURL": "https://ww-save-nyc.firebaseio.com/"
})

## Interfacing with your Firebase Collections

#### Load data to bootstrap

In [4]:
df = pd.read_csv("../../data/superhero_timesheets.csv")
df

,superhero,date,active_on_duty
0,Daredevil,2022-05-29,1
1,Daredevil,2022-05-30,1
2,Daredevil,2022-05-31,1
3,Deadpool,2022-05-30,1
4,Deadpool,2022-05-31,1
5,Dr X,2022-05-29,0
6,Dr X,2022-05-30,1
7,Luke Cage,2022-05-30,0
8,Luke Cage,2022-05-31,0
9,Mr Fantastic,2022-05-31,1


#### Connect to your desired collection

In [5]:
db = firestore.client()
doc_ref = db.collection("superhero_timesheets")

#### Bootstrap all records

In [6]:
%%time
tmp = df.to_dict(orient="records")

list(map(lambda x: doc_ref.add(x), tmp))

CPU times: user 200 ms, sys: 104 ms, total: 304 ms
Wall time: 7.78 s


[(DatetimeWithNanoseconds(2022, 5, 31, 8, 57, 55, 650000, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x122f49760>),
 (DatetimeWithNanoseconds(2022, 5, 31, 8, 57, 55, 934642, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x105e14ee0>),
 (DatetimeWithNanoseconds(2022, 5, 31, 8, 57, 56, 580263, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x122da3e20>),
 (DatetimeWithNanoseconds(2022, 5, 31, 8, 57, 56, 810825, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x122f10e50>),
 (DatetimeWithNanoseconds(2022, 5, 31, 8, 57, 57, 307917, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x122df0e50>),
 (DatetimeWithNanoseconds(2022, 5, 31, 8, 57, 57, 582411, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x122f522e0>),
 (DatetimeWithNanoseco

#### Query to fetch all records for today

In [7]:
today = pd.Timestamp.now().date()
res_0 = pd.DataFrame([
    doc.to_dict() for doc in db.collection(u"superhero_timesheets").where(u"date", u">=", str(today)).stream()
])

res_0

,active_on_duty,superhero,date
0,1,Peeping Tom,2022-05-31
1,0,Luke Cage,2022-05-31
2,1,Spiderman,2022-05-31
3,1,Mr Fantastic,2022-05-31
4,1,Deadpool,2022-05-31
5,1,Daredevil,2022-05-31
6,1,Wolverine,2022-05-31


#### Adding a record

In [8]:
%%time
doc_ref.add({
    "superhero": "Leonardo",
    "active_on_duty": 1,
    "date": "2022-05-31"
})

CPU times: user 5.48 ms, sys: 5.83 ms, total: 11.3 ms
Wall time: 219 ms


(DatetimeWithNanoseconds(2022, 5, 31, 8, 58, 3, 138363, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x122f6b4f0>)

In [9]:
res_1 = pd.DataFrame([
    doc.to_dict() for doc in db.collection(u"superhero_timesheets").where(u"date", u">=", str(today)).stream()
])

res_1

,superhero,date,active_on_duty
0,Peeping Tom,2022-05-31,1
1,Luke Cage,2022-05-31,0
2,Spiderman,2022-05-31,1
3,Leonardo,2022-05-31,1
4,Mr Fantastic,2022-05-31,1
5,Deadpool,2022-05-31,1
6,Daredevil,2022-05-31,1
7,Wolverine,2022-05-31,1


#### Dropping a collection

In [10]:
%%time
def delete_collection(coll_ref, batch_size=200):
    docs = coll_ref.limit(batch_size).stream()
    deleted = 0

    for doc in docs:
        print(f'Deleting doc {doc.id} => {doc.to_dict()}')
        doc.reference.delete()
        deleted = deleted + 1

    if deleted >= batch_size:
        return delete_collection(coll_ref, batch_size)
    
delete_collection(doc_ref)

Deleting doc 0AqvpH6bzbV5Nf54RBfp => {'superhero': 'Peeping Tom', 'active_on_duty': 1, 'date': '2022-05-31'}
Deleting doc 2nQwqMjDbRtqTUc98oRE => {'date': '2022-05-29', 'superhero': 'The Comedian', 'active_on_duty': 1}
Deleting doc 397oZwQ6exWcBF8gjeIw => {'superhero': 'Deadpool', 'date': '2022-05-30', 'active_on_duty': 1}
Deleting doc 39G5lXjOR5G5WgLyX0DH => {'active_on_duty': 0, 'date': '2022-05-31', 'superhero': 'Luke Cage'}
Deleting doc 9DeWpHv84aBLVebEvKqy => {'superhero': 'Spiderman', 'date': '2022-05-31', 'active_on_duty': 1}
Deleting doc A0BTIamYqmhEYPmKy1yB => {'superhero': 'Luke Cage', 'active_on_duty': 0, 'date': '2022-05-30'}
Deleting doc A5dT5iA1SQBkyL7ofTFy => {'superhero': 'Leonardo', 'date': '2022-05-31', 'active_on_duty': 1}
Deleting doc Jx8ntuHehvJCaG53vKYK => {'superhero': 'Mr Fantastic', 'date': '2022-05-31', 'active_on_duty': 1}
Deleting doc LEM1c8JF5mG6dYObklvv => {'active_on_duty': 1, 'superhero': 'Wolverine', 'date': '2022-05-29'}
Deleting doc Om4PKPbP1IuoD4xwlt

## Outro
Thank you for following this workshop all the way to the ends. All credits to [Firebase Docs](https://firebase.google.com/docs) for code snippets used in this notebook.